In [1]:
%reload_ext autoreload
%autoreload 2
from simulation_v3 import Sim
from simulation_v3 import SimTester

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import pandas as pd
import os
import sys
import time


In [29]:
# In matrix form using the new method:

# define the network adjacency matrix
    # Servers/Sources/Queues defined on diagional as:
    # 1 on diagonal: customer arrives
    # -1 on diagonal: server with service time
    # 0 on diagonal: server with no service time
        # customer can only move to a server if not busy

    # Connections between elements of system defined off diagonal as:
    # sum(non-diagonal) = 1:
        # entries are the probability of a customer moving from one element to another
    # sum(non-diagonal) > 1 (ie, each child node has 1 in adj matrix):
        # send customers to all child node with shortest queue
    # sum(non-diagonal) = 0:
        # server leads to a sink

adj_sim    = np.array([ [ 1,  0.5,   0,   0,     0.25,  0.25,   0,    0 ],
                        [ 0, -0.9,  0.6,  0,     0.15,  0.25,   0,    0 ],
                        [ 0,  0,   -0.5,  0,     0,     0,      0,    0 ],
                        [ 0,  0,    0.5,  1,     0,     0.5,    0,    0 ],
                        [ 0,  0,    0,    0,     -0.942, 0,      0.7,  0.3 ], 
                        [ 0,  0.1,  0.3,  0,     0.5,  -0.7432,   0.1,  0 ], 
                        [ 0,  0,    0,    0,     0,     0,    -0.8432,   0 ],
                        [ 0,  0,    0.1,  0,     0.6,   0.1,   0.2,  -0.9543 ],])

# max queue lengths ( not too important with so few customers and long server time for each event)
# can be changed later when mixing more notes and using a larger matrix. 
queue_list = [127,127,127,127,127,127,127,127]

# define the distributions for each server
    # name of distribution, parameters (example: ['exponential', mean=1.0])
distributions = [['normal', 8, 3],
                 ['normal', 0.75, 0.5],
                 ['normal', 0.64, 0.1],
                 ['normal', 0.5, 0.2],
                 ['normal', 5, 3],
                 ['normal', 0.723, 0.1],
                 ['normal', 0.41, 5],
                 ['normal', 0.67, 2],
                 ['normal', 0.67, 2]]

np.random.seed(42)
seeds = np.random.randint(0, 99999, size=1)
sim = Sim(adj_sim, distributions, queue_list, seeds=seeds, generate_log=True, animation=False, record_history=False, logging_mode='Music')

sim.run(number_of_customers=1000)

1: 0.44902801513671875 elapsed time for 466.08178264784436 simulation time with 1000 customers


In [30]:
%reload_ext autoreload
%autoreload 2

from sim_log_process_music import process_adjsim_log

# this file is still very rough.... The note levels are being randomly selected 30-127

# velcoity are based off queue length

# Only using mod 2 as a way to create "quiet" time. Might need this to be a parameter the network outputs such that certain events can be "quiet" and others can be "loud"

# many more tuning improvements to be made - Change the matrix above for highly variable output. 

process_adjsim_log()

In [80]:
# generate a random nxn matrix where the sum of each row except the diagonal is 1
# this will be the adj matrix
# generate a random distribution for each server
size = 32
sources = size // 8
matrix = np.random.rand(size,size)

# randomly select source and sink nodes
sources = np.random.choice(size, sources, replace=False)

# set the values of all columns to 0 for columns that are sources or sinks
for i in sources:
    matrix[:,i] = 0
    matrix[i,i] = 0

for i in [x for x in np.arange(0,size) if x not in sources]:
    matrix[i][i] = 0

# normalize the matrix where the sum of each row is 1 except for the diagonal
for i in range(size):
    matrix[i] = matrix[i] / sum(matrix[i])

# set the values of diagonals according to source and sink nodes
for i in sources:
    matrix[i,i] = 1.0

for i in [x for x in np.arange(0,size) if x not in sources]:
    matrix[i][i] = -1.0

# create a random distribution for each server
distributions = []
for i in range(size):
    distributions.append(['normal', np.random.rand(), np.random.rand()])

# create a random queue list
queue_list = np.random.randint(50, 126, size=size)

# randomly choose the instruments for each server
instruments = np.random.randint(0, 45, size=size)

# check that the matrix is valid by trying to create a simulation and checking for errors
try:
    np.random.seed(42)
    seeds = np.random.randint(0, 99999, size=1)
    sim = Sim(matrix, distributions, queue_list, seeds=seeds, generate_log=True, animation=False, record_history=False, logging_mode='Music')
    sim.run(number_of_customers=1000)
except Exception as e:
    print(e)
    print(e.args)
    print('Invalid matrix')

process_adjsim_log(instruments=instruments)


1: 1.6349973678588867 elapsed time for 97.81787520114237 simulation time with 1000 customers


In [87]:
from midi2audio import FluidSynth
import os

file_path = r'S:\Sadie\2024\475\Project\song-extender\experimenting with adjsims\output.mid'
print(os.path.exists(file_path))

FluidSynth().play_midi(r'S:\Sadie\2024\475\Project\song-extender\experimenting with adjsims\output.mid')


True


FileNotFoundError: [WinError 2] The system cannot find the file specified